In [12]:
from __future__ import print_function
import numpy as np
import os
import re
import nltk 
from nltk.corpus import stopwords 
from nltk.tokenize import word_tokenize, sent_tokenize 
from nltk.corpus import wordnet
stop_words = set(stopwords.words('english')) 
import re
import string
import sys
import unicodedata
import functools
from threading import Thread

In [2]:
def isSynom(word1,word2):
    for syn in wordnet.synsets(word1):
        if word2 in syn.lemma_names():
            return True
    return False
isSynom('sd','sd')

False

In [3]:
def synonyms(word):
    synonyms=[]
    for syn in wordnet.synsets(word):
        for l in syn.lemmas():
            synonyms.append(l.name())
    return synonyms
print(synonyms('total'))

['sum', 'total', 'totality', 'aggregate', 'sum', 'amount', 'total', 'total', 'number', 'add_up', 'come', 'amount', 'total', 'tot', 'tot_up', 'sum', 'sum_up', 'summate', 'tote_up', 'add', 'add_together', 'tally', 'add_up', 'total', 'entire', 'full', 'total', 'full', 'total']


In [4]:
class Column:
    def __init__(self, name='', type=None, equivalences=None):
        self._name = name

        if not type:
            type = []
        self._type = type

        if not equivalences:
            equivalences = []
        self._equivalences = equivalences

        self.primary = False
        self.foreign = False

    @property
    def name(self):
        return self._name

    @property
    def get_type(self):
        return self._type

    def add_type(self, type):
        self.type.append(type)

    @property
    def equivalences(self):
        return self._equivalences

    def add_equivalence(self, equivalence):
        self.equivalences.append(equivalence)

    def is_equivalent(self, word):
        if word in self.equivalences:
            return True
        else:
            return False

    def is_primary(self):
        return self.primary

    def set_as_primary(self):
        self.primary = True

    def is_foreign(self):
        return self.foreign

    def set_as_foreign(self, references):
        self.foreign = references


In [5]:
class Table:
    def __init__(self, name='', columns=None, equivalences=None):
        self._name = name

        if not columns:
            columns = []
        self.columns = columns

        if not equivalences:
            equivalences = []
        self.equivalences = equivalences

    @property
    def name(self):
        return self._name

    @name.setter
    def name(self, value):
        self._name = value

    def get_number_of_columns(self):
        return len(self.columns)

    def get_columns(self):
        return self.columns

    def get_column_by_name(self, column_name):
        for column in self.columns:
            if column.name == column_name:
                return column

    def add_column(self, column_name, column_type, column_equivalences):
        self.columns.append(Column(column_name, column_type, column_equivalences))

    def get_equivalences(self):
        return self.equivalences

    def add_equivalence(self, equivalence):
        self.equivalences.append(equivalence)

    def is_equivalent(self, word):
        if word in self.equivalences:
            return True
        else:
            return False

    def get_primary_keys(self):
        primary_keys = []
        for column in self.columns:
            if column.is_primary():
                primary_keys.append(column)
        return primary_keys

    def get_primary_key_names(self):
        primary_keys = []
        for column in self.columns:
            if column.is_primary():
                primary_keys.append(column.name)
        return primary_keys

    def add_primary_key(self, primary_key_column):
        for column in self.columns:
            if column.name == primary_key_column:
                column.set_as_primary()

    def get_foreign_keys(self):
        foreign_keys = []
        for column in self.columns:
            if column.is_foreign():
                foreign_keys.append(column)
        return foreign_keys

    def get_foreign_key_names(self):
        foreign_keys = []
        for column in self.columns:
            if column.is_foreign():
                foreign_keys.append(column.name)
        return foreign_keys

    def add_foreign_key(self, column_name, foreign_table, foreign_column):
        for column in self.columns:
            if column.name == column_name:
                column.set_as_foreign({'foreign_table': foreign_table, 'foreign_column': foreign_column})

In [6]:
class Database:

    def __init__(self):
        self.tables = []
        self.thesaurus_object = None

    def set_thesaurus(self, thesaurus):
        self.thesaurus_object = thesaurus

    def get_number_of_tables(self):
        return len(self.tables)

    def get_tables(self):
        return self.tables

    def get_column_with_this_name(self, name):
        for table in self.tables:
            for column in table.get_columns():
                if column.name == name:
                    return column

    def get_table_by_name(self, table_name):
        for table in self.tables:
            if table.name == table_name:
                return table

    def get_tables_into_dictionary(self):
        data = {}
        for table in self.tables:
            data[table.name] = []
            for column in table.get_columns():
                data[table.name].append(column.name)
        return data

    def get_primary_keys_by_table(self):
        data = {}
        for table in self.tables:
            data[table.name] = table.get_primary_keys()
        return data

    def get_foreign_keys_by_table(self):
        data = {}
        for table in self.tables:
            data[table.name] = table.get_foreign_keys()
        return data

    def get_primary_keys_of_table(self, table_name):
        for table in self.tables:
            if table.name == table_name:
                return table.get_primary_keys()

    def get_primary_key_names_of_table(self, table_name):
        for table in self.tables:
            if table.name == table_name:
                return table.get_primary_key_names()

    def get_foreign_keys_of_table(self, table_name):
        for table in self.tables:
            if table.name == table_name:
                return table.get_foreign_keys()

    def get_foreign_key_names_of_table(self, table_name):
        for table in self.tables:
            if table.name == table_name:
                return table.get_foreign_key_names()

    def add_table(self, table):
        self.tables.append(table)

    @staticmethod
    def _generate_path(path):
        cwd = os.path.dirname(__file__)
        filename = os.path.join(cwd, path)
        return filename

    def load(self, path):
        with open(path) as f:
            content = f.read()
            tables_string = [p.split(';')[0] for p in content.split('CREATE') if ';' in p]
            for table_string in tables_string:
                if 'TABLE' in table_string:
                    table = self.create_table(table_string)
                    self.add_table(table)
            alter_tables_string = [p.split(';')[0] for p in content.split('ALTER') if ';' in p]
            for alter_table_string in alter_tables_string:
                if 'TABLE' in alter_table_string:
                    self.alter_table(alter_table_string)

    def predict_type(self, string):
        if 'int' in string.lower():
            return 'int'
        elif 'char' in string.lower() or 'text' in string.lower():
            return 'string'
        elif 'date' in string.lower():
            return 'date'
        else:
            return 'unknow'

    def create_table(self, table_string):
        lines = table_string.split("\n")
        table = Table()
        for line in lines:
            if 'TABLE' in line:
                table_name = re.search("`(\w+)`", line)
                table.name = table_name.group(1)
                if self.thesaurus_object is not None:
                    table.equivalences = self.thesaurus_object.get_synonyms_of_a_word(table.name)
            elif 'PRIMARY KEY' in line:
                primary_key_columns = re.findall("`(\w+)`", line)
                for primary_key_column in primary_key_columns:
                    table.add_primary_key(primary_key_column)
            else:
                column_name = re.search("`(\w+)`", line)
                if column_name is not None:
                    column_type = self.predict_type(line)
                    if self.thesaurus_object is not None:
                        equivalences = self.thesaurus_object.get_synonyms_of_a_word(column_name.group(1))
                    else:
                        equivalences = []
                    table.add_column(column_name.group(1), column_type, equivalences)
        return table

    def alter_table(self, alter_string):
        lines = alter_string.replace('\n', ' ').split(';')
        for line in lines:
            if 'PRIMARY KEY' in line:
                table_name = re.search("TABLE `(\w+)`", line).group(1)
                table = self.get_table_by_name(table_name)
                primary_key_columns = re.findall("PRIMARY KEY \(`(\w+)`\)", line)
                for primary_key_column in primary_key_columns:
                    table.add_primary_key(primary_key_column)
            elif 'FOREIGN KEY' in line:
                table_name = re.search("TABLE `(\w+)`", line).group(1)
                table = self.get_table_by_name(table_name)
                foreign_keys_list = re.findall("FOREIGN KEY \(`(\w+)`\) REFERENCES `(\w+)` \(`(\w+)`\)", line)
                for column, foreign_table, foreign_column in foreign_keys_list:
                    table.add_foreign_key(column, foreign_table, foreign_column)


In [7]:
def changeQuery(database_path,line):
    database =Database()
    database.load(database_path)
    dict_database = database.get_tables_into_dictionary()
    data = {'sentence':'', 'count':'','select':'', 'from':'','where': ''}
    data['sentence'] =line
    words = word_tokenize(line)
    word_dict=nltk.pos_tag(words)
    print(word_dict)
    for i in range(len(word_dict)):
        if (word_dict[i][-1]== 'WRB'):
            print('count procedding')
            for j in range (i,len(word_dict)):
                    if (word_dict[j][-1][0]== 'N'):
                        draft = word_dict[j][0]
                        if(database.get_column_with_this_name(draft)!= None):
                            data['count'] = draft
                        elif(isSynom(database.get_column_with_this_name(draft).name,draft)):
                            data['count'] = database.get_column_with_this_name(draft)
                        break
            break
        if ((word_dict[i][-1][0]== 'W') and (data['count'] == '')):
            print('select procedding...')
            for j in range (i,len(word_dict)):
                    if (word_dict[j][-1][0]== 'N'):
                        draft = word_dict[j][0]
                        if(database.get_column_with_this_name(draft)!= None):
                            data['select'] = draft
                        elif(isSynom(database.get_column_with_this_name(draft).name,draft)):
                            data['select'] = database.get_column_with_this_name(draft)
                        break
            break
    for i in range(len(word_dict)):
        if (word_dict[i][-1][0]== 'N') or (word_dict[i][-1]== 'JJR'):
            if (word_dict[i][0]) in dict_database.keys():
                data['from']=word_dict[i][0]
            else:
                for key in dict_database.keys():
                    if isSynom(key,word_dict[i][0]):
                        data['from']= key
                                    
                
    for i in range(len(word_dict)):
        data['where']=[]
        if (word_dict[i][-1][0]== 'N') or (word_dict[i][-1]== 'JJR'):
            if(database.get_column_with_this_name(word_dict[i][0])!= None):
                if data['select'] != word_dict[i][0]:
                    data['where'].append(word_dict[i][0])
                     for j in range (i,len(word_dict)):
                            if (word_dict[i][-1])=''
                    
                
    
    sql ='count '+data['count']+' Select '+data['select']+'\n'+' from '+data['from']+'\n'+ ' where '+data['where']
    return sql
    
            
    

IndentationError: unexpected indent (<ipython-input-7-28df72d749f2>, line 49)

In [8]:
print(changeQuery('database_store/city.sql','what many the id of the city which name is jaffna'))

NameError: name 'changeQuery' is not defined

In [7]:
def get_table(database,phrase):
    table =[]
    dict_database = database.get_tables_into_dictionary()
    words = word_tokenize(phrase)
    filtered_words = [word for word in words if word not in stop_words]
    for word in filtered_words:
        if word in dict_database.keys():
            table.append(word)
    return table

In [8]:
def predictColumn(database,table_name,phrase):
    columns =[]
    dict_database = database.get_tables_into_dictionary()
    #print(dict_database)
    #print(table_name)
    words = word_tokenize(phrase)
    filtered_words = [word for word in words if word not in stop_words]

    word_dict=nltk.pos_tag(filtered_words)
    #print(word_dict)
    table = dict_database[table_name]
    for j in range (len(word_dict)):
        if (word_dict[j][-1][0]== 'N') or (word_dict[j][-1][0]== 'J'):
            sample = word_dict[j][0]
            #print(sample)
            for name in table:
                if name == sample:
                    columns.append(name)
                elif isSynom(sample,name):
                    columns.append(name)
    return columns

In [23]:
def sumElements(dict):
    c=0
    for item in dict.keys():
        c=c+1
    return c

In [53]:
def parsingPhrase(database,phrase):
    tables=get_table(database,phrase)
    table=tables[0]
    words = word_tokenize(phrase)
    im_words={}
    for i in range(len(words)):
        tag = nltk.tag.pos_tag([words[i]])
        if tag[0][-1][0]=='W':
            im_words[words[i]]=i
        if words[i]== table:
            im_words['table'] =i
    return im_words

In [57]:
def selectColumnParser(database,phrase):
    data={'select':'','count':'','max':'','min':'','sum':'','distinct':'','error':'','avg':''}
    tables=get_table(database,phrase)
    table=''
    #print(tables)
    if(len(tables) ==0):
        print('you have no tables in your phrase')
    elif(len(tables) ==1):
        table=tables[0]
    if table != '':
        #print(table)
        columns = predictColumn(database,table,phrase)
        #print(columns)
        words = word_tokenize(phrase)
        im_words=parsingPhrase(database,phrase)
        for i in range(len(words)):
            if words[i] in synonyms('count') and (words[i] not in columns):
                for j in range(i,int(im_words['table'])):
                    if words[j] in columns:
                        data['count']= words[j]
                        break
                    else:
                        data['count']= '*'
                    
            elif words[i] in synonyms('total') and (words[i] not in columns):
                for j in range(i,int(im_words['table'])):
                    if words[j] in columns:
                        data['sum']= words[j]
                        break
                    else:
                        data['sum']= '*'
            if words[i] in synonyms('maximum') and (words[i] not in columns):
                for j in range(i,int(im_words['table'])):
                    if words[j] in columns:
                        data['max']= words[j]
                        break
                    else:
                        data['max']= '*'
            if words[i] in synonyms('minimum') and (words[i] not in columns):
                for j in range(i,int(im_words['table'])):
                    if words[j] in columns:
                        data['min']= words[j]
                        break
                    else:
                        data['min']= '*'
            if words[i] in synonyms('distinct') and (words[i] not in columns):
                for j in range(i,int(im_words['table'])):
                    if words[j] in columns:
                        data['distinct']= words[j]
                        break
                    else:
                        data['distinct']= '*'
            if words[i] in synonyms('average') and (words[i] not in columns):
                for j in range(i,int(im_words['table'])):
                    if words[j] in columns:
                        data['avg']= words[j]
                        break
                    else:
                        data['avg']= '*'
        print(data)
        for word in words:
            if im_words['table']:
                index_of_the_table= im_words.get('table')
                if sumElements(im_words)==2:
                    for item in im_words:
                        item.value()
    
        
        
    
        

In [9]:
database=Database()
database.load('database_store/city.sql')


In [33]:
class SelectParser(Thread):
    def __init__(self, columns_of_select, tables_of_from, phrase, database):
        Thread.__init__(self)
        self.select_objects = []
        self.columns_of_select = columns_of_select
        self.tables_of_from = tables_of_from
        self.phrase = phrase
        self.count_keywords =str('number, how many, count')
        self.sum_keywords =str('sum, total').split(',')
        self.average_keywords = str('average, avg, Average').split(',')
        self.max_keywords =str('maximum, highest, max').split(',')
        self.min_keywords =str('minimum, lowest, min').split(',')
        self.distinct_keywords =str('distinct, different, distinctive, distinctly, unique').split(',')
        self.database_object = database
        self.database_dico = self.database_object.get_tables_into_dictionary()
    

        

    def get_tables_of_column(self, column):
        tmp_table = []
        for table in self.database_dico:
            if column in self.database_dico[table]:
                tmp_table.append(table)
        return tmp_table

    def get_column_name_with_alias_table(self, column, table_of_from):
        one_table_of_column = self.get_tables_of_column(column)[0]
        tables_of_column = self.get_tables_of_column(column)
        if table_of_from in tables_of_column:
            return str(table_of_from) + '.' + str(column)
        else:
            return str(one_table_of_column) + '.' + str(column)

    def uniquify(self, list):
        already = []
        for element in list:
            if element not in already:
                already.append(element)
        return already

    def run(self):
        for table_of_from in self.tables_of_from:  # for each query
            self.select_object = Select()
            is_count = False
            self.columns_of_select = self.uniquify(self.columns_of_select)
            number_of_select_column = len(self.columns_of_select)

            if number_of_select_column == 0:
                select_type = []
                for count_keyword in self.count_keywords:
                    # if count_keyword in (word.lower() for word in self.phrase):
                    # so that it matches multiple words too in keyword synonymn in .lang rather than just single word for COUNT
                    # (e.g. QUERY-> "how many city there are in which the employe name is aman ?" )
                    lower_self_phrase = ' '.join(word.lower() for word in self.phrase)
                    if count_keyword in lower_self_phrase:
                        select_type.append('COUNT')

                self.select_object.add_column(None, self.uniquify(select_type))
            else:
                select_phrases = []
                previous_index = 0

                for i in range(0, len(self.phrase)):
                    for column_name in self.columns_of_select:
                        if (self.phrase[i] == column_name) or (
                                    self.phrase[i] in self.database_object.get_column_with_this_name(column_name).equivalences):
                            select_phrases.append(self.phrase[previous_index:i + 1])
                            previous_index = i + 1

                select_phrases.append(self.phrase[previous_index:])

                for i in range(0, len(select_phrases)):  # for each select phrase (i.e. column processing)
                    select_type = []

                    phrase = [word.lower() for word in select_phrases[i]]

                    for keyword in self.average_keywords:
                        if keyword in phrase:
                            select_type.append('AVG')
                    for keyword in self.count_keywords:
                        if keyword in phrase:
                            select_type.append('COUNT')
                    for keyword in self.max_keywords:
                        if keyword in phrase:
                            select_type.append('MAX')
                    for keyword in self.min_keywords:
                        if keyword in phrase:
                            select_type.append('MIN')
                    for keyword in self.sum_keywords:
                        if keyword in phrase:
                            select_type.append('SUM')
                    for keyword in self.distinct_keywords:
                        if keyword in phrase:
                            select_type.append('DISTINCT')

                    if (i != len(select_phrases) - 1):
                        column = self.get_column_name_with_alias_table(self.columns_of_select[i], table_of_from)
                        self.select_object.add_column(column, self.uniquify(select_type))

            self.select_objects.append(self.select_object)

    def join(self):
        Thread.join(self)
        return self.select_objects


In [ ]:
class WhereParser(Thread):
    def __init__(self, phrases, tables_of_from, columns_of_values_of_where, count_keywords, sum_keywords,
                 average_keywords, max_keywords, min_keywords, greater_keywords, less_keywords, between_keywords,
                 negation_keywords, junction_keywords, disjunction_keywords, like_keywords, distinct_keywords,
                 database_dico, database_object):
        Thread.__init__(self)
        self.where_objects = []
        self.phrases = phrases
        self.tables_of_from = tables_of_from
        self.columns_of_values_of_where = columns_of_values_of_where
        self.count_keywords = count_keywords
        self.sum_keywords = sum_keywords
        self.average_keywords = average_keywords
        self.max_keywords = max_keywords
        self.min_keywords = min_keywords
        self.greater_keywords = greater_keywords
        self.less_keywords = less_keywords
        self.between_keywords = between_keywords
        self.negation_keywords = negation_keywords
        self.junction_keywords = junction_keywords
        self.disjunction_keywords = disjunction_keywords
        self.like_keywords = like_keywords
        self.distinct_keywords = distinct_keywords
        self.database_dico = database_dico
        self.database_object = database_object

    def get_tables_of_column(self, column):
        tmp_table = []
        for table in self.database_dico:
            if column in self.database_dico[table]:
                tmp_table.append(table)
        return tmp_table

    def get_column_name_with_alias_table(self, column, table_of_from):
        one_table_of_column = self.get_tables_of_column(column)[0]
        tables_of_column = self.get_tables_of_column(column)
        if table_of_from in tables_of_column:
            return str(table_of_from) + '.' + str(column)
        else:
            return str(one_table_of_column) + '.' + str(column)

    def intersect(self, a, b):
        return list(set(a) & set(b))

    def predict_operation_type(self, previous_column_offset, current_column_offset):
        interval_offset = list(range(previous_column_offset, current_column_offset))
        if (len(self.intersect(interval_offset, self.count_keyword_offset)) >= 1):
            return 'COUNT'
        elif (len(self.intersect(interval_offset, self.sum_keyword_offset)) >= 1):
            return 'SUM'
        elif (len(self.intersect(interval_offset, self.average_keyword_offset)) >= 1):
            return 'AVG'
        elif (len(self.intersect(interval_offset, self.max_keyword_offset)) >= 1):
            return 'MAX'
        elif (len(self.intersect(interval_offset, self.min_keyword_offset)) >= 1):
            return 'MIN'
        else:
            return None

    def predict_operator(self, current_column_offset, next_column_offset):
        interval_offset = list(range(current_column_offset, next_column_offset))

        if (len(self.intersect(interval_offset, self.negation_keyword_offset)) >= 1) and (
                    len(self.intersect(interval_offset, self.greater_keyword_offset)) >= 1):
            return '<'
        elif (len(self.intersect(interval_offset, self.negation_keyword_offset)) >= 1) and (
                    len(self.intersect(interval_offset, self.less_keyword_offset)) >= 1):
            return '>'
        if (len(self.intersect(interval_offset, self.less_keyword_offset)) >= 1):
            return '<'
        elif (len(self.intersect(interval_offset, self.greater_keyword_offset)) >= 1):
            return '>'
        elif (len(self.intersect(interval_offset, self.between_keyword_offset)) >= 1):
            return 'BETWEEN'
        elif (len(self.intersect(interval_offset, self.negation_keyword_offset)) >= 1):
            return '!='
        elif (len(self.intersect(interval_offset, self.like_keyword_offset)) >= 1):
            return 'LIKE'
        else:
            return '='

    def predict_junction(self, previous_column_offset, current_column_offset):
        interval_offset = list(range(previous_column_offset, current_column_offset))
        junction = 'AND'
        if (len(self.intersect(interval_offset, self.disjunction_keyword_offset)) >= 1):
            return 'OR'
        elif (len(self.intersect(interval_offset, self.junction_keyword_offset)) >= 1):
            return 'AND'

        first_encountered_junction_offset = -1
        first_encountered_disjunction_offset = -1

        for offset in self.junction_keyword_offset:
            if offset >= current_column_offset:
                first_encountered_junction_offset = offset
                break

        for offset in self.disjunction_keyword_offset:
            if offset >= current_column_offset:
                first_encountered_disjunction_offset = offset
                break

        if first_encountered_junction_offset >= first_encountered_disjunction_offset:
            return 'AND'
        else:
            return 'OR'

    def uniquify(self, list):
        already = []
        for element in list:
            if element not in already:
                already.append(element)
        return already

    def run(self):
        number_of_where_columns = 0
        columns_of_where = []
        offset_of = {}
        column_offset = []
        self.count_keyword_offset = []
        self.sum_keyword_offset = []
        self.average_keyword_offset = []
        self.max_keyword_offset = []
        self.min_keyword_offset = []
        self.greater_keyword_offset = []
        self.less_keyword_offset = []
        self.between_keyword_offset = []
        self.junction_keyword_offset = []
        self.disjunction_keyword_offset = []
        self.negation_keyword_offset = []
        self.like_keyword_offset = []

        for phrase in self.phrases:
            phrase_offset_string = ''
            for i in range(0, len(phrase)):
                for table_name in self.database_dico:
                    columns = self.database_object.get_table_by_name(table_name).get_columns()
                    for column in columns:
                        if (phrase[i] == column.name) or (phrase[i] in column.equivalences):
                            number_of_where_columns += 1
                            columns_of_where.append(column.name)
                            offset_of[phrase[i]] = i
                            column_offset.append(i)
                            break
                    else:
                        continue
                    break

                phrase_keyword = str(phrase[i]).lower()  # for robust keyword matching
                phrase_offset_string += phrase_keyword + " "

                for keyword in self.count_keywords:
                    if keyword in phrase_offset_string :    # before the column
                        if (phrase_offset_string.find(keyword) + len(keyword) + 1 == len(phrase_offset_string) ) :
                            self.count_keyword_offset.append(i)

                for keyword in self.sum_keywords:
                    if keyword in phrase_offset_string :    # before the column
                        if (phrase_offset_string.find(keyword) + len(keyword) + 1 == len(phrase_offset_string) ) :
                            self.sum_keyword_offset.append(i)

                for keyword in self.average_keywords:
                    if keyword in phrase_offset_string :    # before the column
                        if (phrase_offset_string.find(keyword) + len(keyword) + 1 == len(phrase_offset_string) ) :
                            self.average_keyword_offset.append(i)

                for keyword in self.max_keywords:
                    if keyword in phrase_offset_string :    # before the column
                        if (phrase_offset_string.find(keyword) + len(keyword) + 1 == len(phrase_offset_string) ) :
                            self.max_keyword_offset.append(i)

                for keyword in self.min_keywords:
                    if keyword in phrase_offset_string :    # before the column
                        if (phrase_offset_string.find(keyword) + len(keyword) + 1 == len(phrase_offset_string) ) :
                            self.min_keyword_offset.append(i)

                for keyword in self.greater_keywords:
                    if keyword in phrase_offset_string :    # after the column
                        if (phrase_offset_string.find(keyword) + len(keyword) + 1 == len(phrase_offset_string) ) :
                            self.greater_keyword_offset.append(i)

                for keyword in self.less_keywords:
                    if keyword in phrase_offset_string :    # after the column
                        if (phrase_offset_string.find(keyword) + len(keyword) + 1 == len(phrase_offset_string) ) :
                            self.less_keyword_offset.append(i)

                for keyword in self.between_keywords:
                    if keyword in phrase_offset_string :    # after the column
                        if (phrase_offset_string.find(keyword) + len(keyword) + 1 == len(phrase_offset_string) ) :
                            self.between_keyword_offset.append(i)

                for keyword in self.junction_keywords:
                    if keyword in phrase_offset_string :    # after the column
                        if (phrase_offset_string.find(keyword) + len(keyword) + 1 == len(phrase_offset_string) ) :
                            self.junction_keyword_offset.append(i)

                for keyword in self.disjunction_keywords:
                    if keyword in phrase_offset_string :    # after the column
                        if (phrase_offset_string.find(keyword) + len(keyword) + 1 == len(phrase_offset_string) ) :
                            self.disjunction_keyword_offset.append(i)

                for keyword in self.negation_keywords:
                    if keyword in phrase_offset_string :
                        if (phrase_offset_string.find(keyword) + len(keyword) + 1 == len(phrase_offset_string) ) :
                            self.negation_keyword_offset.append(i)

                for keyword in self.like_keywords:
                    if keyword in phrase_offset_string :    # after the column
                        if (phrase_offset_string.find(keyword) + len(keyword) + 1 == len(phrase_offset_string) ) :
                            self.like_keyword_offset.append(i)


        for table_of_from in self.tables_of_from:
            where_object = Where()
            for i in range(0, len(column_offset)):
                current = column_offset[i]

                if i == 0:
                    previous = 0
                else:
                    previous = column_offset[i - 1]

                if i == (len(column_offset) - 1):
                    _next = 999
                else:
                    _next = column_offset[i + 1]

                junction = self.predict_junction(previous, current)
                column = self.get_column_name_with_alias_table(columns_of_where[i], table_of_from)
                operation_type = self.predict_operation_type(previous, current)

                if len(self.columns_of_values_of_where) > i:
                    value = self.columns_of_values_of_where[
                        len(self.columns_of_values_of_where) - len(columns_of_where) + i]
                else:
                    value = 'OOV'  # Out Of Vocabulary: default value

                operator = self.predict_operator(current, _next)
                where_object.add_condition(junction, Condition(column, operation_type, operator, value))
            self.where_objects.append(where_object)

    def join(self):
        Thread.join(self)
        return self.where_objects


In [31]:
def parsingSentence(database,phrase):
    dict_database = database.get_tables_into_dictionary()
    selectPhrase=''
    fromPhrase=''
    wherePhrase=''
    table=[]
    columns_of_select=[]
    columns_of_where=[]
    number_of_table=0
    no_of_columns_select=0
    number_of_where_column=0
    last_table_position = 0
    
    columns_of_select=[]
    input_for_finding_value = phrase.rstrip(string.punctuation.replace('"', '').replace("'", ""))
    filter_list = [",", "!"]
    for filter_element in filter_list:
        input_for_finding_value = input_for_finding_value.replace(filter_element, " ")
    input_word_list = input_for_finding_value.split()
    for i in range(0, len(input_word_list)):
        for table_name in dict_database.keys() :
            if (input_word_list[i] == table_name):
                table.append(table_name)
                if number_of_table== 0:
                    select_phrase = input_word_list[:i]
                    table.append(table_name)
                    number_of_table += 1
                    last_table_position = i
            columns = database.get_table_by_name(table_name).get_columns()
            for column in columns:
                if (input_word_list[i] == column.name):
                    if number_of_table == 0:
                        columns_of_select.append(column.name)
                        no_of_columns_select=no_of_columns_select+1
                    else:
                        if number_of_where_column == 0:
                            from_phrase = input_word_list[len(select_phrase):last_table_position + 1]
                        columns_of_where.append(column.name)
                        number_of_where_column+=1
                else:
                    if (number_of_table != 0) and (number_of_where_column == 0) and (i == (len(input_word_list) - 1)):
                        from_phrase=input_word_list[len(select_phrase):]
    where_phrase = input_word_list[len(select_phrase) + len(from_phrase):]
    if (number_of_select_column + number_of_table + number_of_where_column) == 0:
        print(no_keywords_found)
    
    
    print(select_phrase,from_phrase,where_phrase)
    print(table)
    print(columns_of_select)
    print(last_table_position)

In [34]:
parsingSentence( database,'give the number of id in city where the cityName is colombo')



['give', 'the', 'number', 'of', 'id', 'in'] ['city'] ['where', 'the', 'cityName', 'is', 'colombo']
['city', 'city']
['id', 'id']
6


In [40]:
type(('average, avg, Average').split(','))

list